# Day 16: Packet Decoder

## Data

In [87]:
puzzleData = "E20D79005573F71DA0054E48527EF97D3004653BB1FC006867A8B1371AC49C801039171941340066E6B99A6A58B8110088BA008CE6F7893D4E6F7893DCDCFDB9D6CBC4026FE8026200DC7D84B1C00010A89507E3CCEE37B592014D3C01491B6697A83CB4F59E5E7FFA5CC66D4BC6F05D3004E6BB742B004E7E6B3375A46CF91D8C027911797589E17920F4009BE72DA8D2E4523DCEE86A8018C4AD3C7F2D2D02C5B9FF53366E3004658DB0012A963891D168801D08480485B005C0010A883116308002171AA24C679E0394EB898023331E60AB401294D98CA6CD8C01D9B349E0A99363003E655D40289CBDBB2F55D25E53ECAF14D9ABBB4CC726F038C011B0044401987D0BE0C00021B04E2546499DE824C015B004A7755B570013F2DD8627C65C02186F2996E9CCD04E5718C5CBCC016B004A4F61B27B0D9B8633F9344D57B0C1D3805537ADFA21F231C6EC9F3D3089FF7CD25E5941200C96801F191C77091238EE13A704A7CCC802B3B00567F192296259ABD9C400282915B9F6E98879823046C0010C626C966A19351EE27DE86C8E6968F2BE3D2008EE540FC01196989CD9410055725480D60025737BA1547D700727B9A89B444971830070401F8D70BA3B8803F16A3FC2D00043621C3B8A733C8BD880212BCDEE9D34929164D5CB08032594E5E1D25C0055E5B771E966783240220CD19E802E200F4588450BC401A8FB14E0A1805B36F3243B2833247536B70BDC00A60348880C7730039400B402A91009F650028C00E2020918077610021C00C1002D80512601188803B4000C148025010036727EE5AD6B445CC011E00B825E14F4BBF5F97853D2EFD6256F8FFE9F3B001420C01A88915E259002191EE2F4392004323E44A8B4C0069CEF34D304C001AB94379D149BD904507004A6D466B618402477802E200D47383719C0010F8A507A294CC9C90024A967C9995EE2933BA840"

In [1]:
testData1 = "8A004A801A8002F478"

In [78]:
testData2 = "620080001611562C8802118E34"

In [3]:
testData3 = "C0015000016115A2E0802F182340"

In [4]:
testData4 = "A0016C880162017C3686B18A3D4780"

## Parse data

In [152]:
from timeit import default_timer as timer

def parseData(data):
    start = timer()
    bitstream = hexToBin(data)
    rootPacket, foo = parsePacket(bitstream)
    end = timer()
    print("parse time: "+"{:10.7f}".format(end-start))
    return rootPacket

def hexToBin(hexstream):
    bitstream = "{0:b}".format(int(hexstream,16),2)
    return bitstream.zfill(len(hexstream)*4)

def parsePacket(bitstream):
    version = int(bitstream[:3],2)
    type = int(bitstream[3:6],2)
    packet = Packet(version,type)
    
    bitstream = bitstream[6:]
    
    if type is 4:
        foundLastChunk = False
        numberbits = ''
        # discard leading 0s
        while not foundLastChunk:
            chunk = bitstream[:5]
            bitstream = bitstream[5:]
            numberbits += chunk[1:]
            if chunk[0] == '0':
                foundLastChunk = True
        packet.setNumber(int(numberbits,2))
    else:
        if bitstream[0] == '0':
            subpacketsLength = int(bitstream[1:16],2)
            subpackets = bitstream[16:subpacketsLength+16]
            bitstream = bitstream[subpacketsLength+16:]
            while len(subpackets) != 0 and not all(x == '0' for x in subpackets):
                subpacket, subpackets = parsePacket(subpackets)
                packet.addChildPacket(subpacket)
        else:
            subpacketsCount = int(bitstream[1:12])
            bitstream = bitstream[12:]
            for _ in range(subpacketsCount):
                if len(bitstream) < 7:
                    break
                subpacket, bitstream = parsePacket(bitstream)
                packet.addChildPacket(subpacket)
    return packet, bitstream

In [169]:
class Packet:
    def __init__(self,v,t):
        self.version = v    
        self.type = t
        self.packets = []
        self.number = 0
    
    def addChildPacket(self, packet):
        self.packets.append(packet)
    
    def setNumber(self, n):
        self.number = n

## Part 1

In [154]:
def sumAllVersionNumbers(data):
    rootNode = parseData(data)
    start = timer()
    versionNumbers = getVersionNumbers(rootNode)
    total = sum(versionNumbers)
    end = timer()
    print("run time: "+"{:10.7f}".format(end-start))
    print(total)
    
def getVersionNumbers(node):
    versionNumbers = [node.version]
    if len(node.packets) > 0:
        for packet in node.packets:
            versionNumbers.extend(getVersionNumbers(packet))
    return versionNumbers

In [130]:
sumAllVersionNumbers(testData1)

parse time:  0.0000735
run time:  0.0000112
16


In [155]:
sumAllVersionNumbers(testData2)

parse time:  0.0000584
run time:  0.0000088
12


In [92]:
sumAllVersionNumbers(testData3)

parse time:  0.0001019
run time:  0.0000153
23


In [93]:
sumAllVersionNumbers(testData4)

parse time:  0.0000468
run time:  0.0000076
31


In [94]:
sumAllVersionNumbers(puzzleData)

parse time:  0.0025020
run time:  0.0003694
951


## Part 2

In [167]:
import numpy

def resolveTheMessage(data):
    rootNode = parseData(data)
    printNodes(rootNode)
    start = timer()
    result = resolveNode(rootNode)
    end = timer()
    print("run time: "+"{:10.7f}".format(end-start))
    print(result)

def resolveNode(node):
    print(node.type, node.version)
    if node.type == 0:
        return sum([resolveNode(packet) for packet in node.packets])
    if node.type == 1:
        return int(numpy.prod([resolveNode(packet) for packet in node.packets]))
    if node.type == 2:
        return min([resolveNode(packet) for packet in node.packets])
    if node.type == 3:
        return max([resolveNode(packet) for packet in node.packets])
    if node.type == 4:
        return node.number
    if node.type == 5:
        return 1 if resolveNode(node.packets[0]) > resolveNode(node.packets[1]) else 0
    if node.type == 6:
        return 1 if resolveNode(node.packets[0]) < resolveNode(node.packets[1]) else 0
    if node.type == 7:
        return 1 if resolveNode(node.packets[0]) == resolveNode(node.packets[1]) else 0

In [163]:
resolveTheMessage("C200B40A82")

parse time:  0.0000472
run time:  0.0000093
3


In [164]:
resolveTheMessage("04005AC33890")

parse time:  0.0000283
run time:  0.0000406
54


In [123]:
resolveTheMessage("880086C3E88112")

parse time:  0.0000485
run time:  0.0000082
7


In [124]:
resolveTheMessage("CE00C43D881120")

parse time:  0.0000431
run time:  0.0000083
9


In [126]:
resolveTheMessage("D8005AC2A8F0")

parse time:  0.0000415
run time:  0.0000054
1


In [127]:
resolveTheMessage("F600BC2D8F")

parse time:  0.0000468
run time:  0.0000066
0


In [128]:
resolveTheMessage("9C005AC2F8F0")

parse time:  0.0000504
run time:  0.0000071
0


In [170]:
resolveTheMessage("9C0141080250320F1802104A08")

parse time:  0.0000413
 version:4 type:7 children:1 number: 0
- version:2 type:0 children:3 number: 0
-- version:2 type:4 children:0 number: 1
-- version:4 type:4 children:0 number: 3
-- version:6 type:1 children:2 number: 0
--- version:0 type:4 children:0 number: 2
--- version:2 type:4 children:0 number: 2
7 4
0 2
4 2
4 4
1 6
4 0
4 2


IndexError: list index out of range

In [159]:
resolveTheMessage(puzzleData)

parse time:  0.0034489
 version:7 type:0 children:2
- version:7 type:1 children:2
-- version:3 type:4 children:0
-- version:5 type:5 children:2
--- version:3 type:4 children:0
--- version:4 type:4 children:0
- version:5 type:1 children:8
-- version:1 type:4 children:0
-- version:7 type:6 children:2
--- version:1 type:4 children:0
--- version:0 type:4 children:0
-- version:7 type:1 children:2
--- version:3 type:4 children:0
--- version:1 type:4 children:0
-- version:3 type:2 children:1
--- version:3 type:4 children:0
-- version:2 type:1 children:1
--- version:5 type:6 children:3
---- version:1 type:4 children:0
---- version:2 type:4 children:0
---- version:3 type:4 children:0
-- version:6 type:1 children:1
--- version:7 type:6 children:1
---- version:3 type:0 children:5
----- version:3 type:4 children:0
----- version:5 type:4 children:0
----- version:5 type:4 children:0
----- version:0 type:0 children:3
------ version:2 type:4 children:0
------ version:2 type:4 children:0
------ version

IndexError: list index out of range

## Utilities

In [165]:
def printNodes(rootNode, prepend = ''):
    print(prepend+" version:{} type:{} children:{} number: {}".format(rootNode.version, rootNode.type, len(rootNode.packets), rootNode.number))
    [printNodes(node, prepend+'-') for node in rootNode.packets]